In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection  import train_test_split
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from sklearn.cluster import AgglomerativeClustering 

from scipy.stats import zscore

Load excel sheet into pandas

In [5]:
df = pd.read_excel('Credit Card Customer Data.xlsx')

View first 10 rows

In [6]:
df.head(10)

,Sl_No,Customer Key,Avg_Credit_Limit,Total_Credit_Cards,Total_visits_bank,Total_visits_online,Total_calls_made
0,1,87073,100000,2,1,1,0
1,2,38414,50000,3,0,10,9
2,3,17341,50000,7,1,3,4
3,4,40496,30000,5,1,1,4
4,5,47437,100000,6,0,12,3
5,6,58634,20000,3,0,1,8
6,7,48370,100000,5,0,11,2
7,8,37376,15000,3,0,1,1
8,9,82490,5000,2,0,2,2
9,10,44770,3000,4,0,1,7


we need to drop the serial number as its not useful

In [7]:
df = df.drop(['Sl_No'], axis=1)

View first and last rows

In [8]:
df.head()

,Customer Key,Avg_Credit_Limit,Total_Credit_Cards,Total_visits_bank,Total_visits_online,Total_calls_made
0,87073,100000,2,1,1,0
1,38414,50000,3,0,10,9
2,17341,50000,7,1,3,4
3,40496,30000,5,1,1,4
4,47437,100000,6,0,12,3


In [9]:
df.tail()

,Customer Key,Avg_Credit_Limit,Total_Credit_Cards,Total_visits_bank,Total_visits_online,Total_calls_made
655,51108,99000,10,1,10,0
656,60732,84000,10,1,13,2
657,53834,145000,8,1,9,1
658,80655,172000,10,1,15,0
659,80150,167000,9,0,12,2


Inspect the data type

In [10]:
df.dtypes

Customer Key           int64
Avg_Credit_Limit       int64
Total_Credit_Cards     int64
Total_visits_bank      int64
Total_visits_online    int64
Total_calls_made       int64
dtype: object

All the features have integer values based on the above output

Check for missing values

In [11]:
df.isnull().any()

Customer Key           False
Avg_Credit_Limit       False
Total_Credit_Cards     False
Total_visits_bank      False
Total_visits_online    False
Total_calls_made       False
dtype: bool

From the output above, we have no null value

Next we check the info

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 660 entries, 0 to 659
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   Customer Key         660 non-null    int64
 1   Avg_Credit_Limit     660 non-null    int64
 2   Total_Credit_Cards   660 non-null    int64
 3   Total_visits_bank    660 non-null    int64
 4   Total_visits_online  660 non-null    int64
 5   Total_calls_made     660 non-null    int64
dtypes: int64(6)
memory usage: 31.1 KB


Check for unique values

In [13]:
df.nunique()

Customer Key           655
Avg_Credit_Limit       110
Total_Credit_Cards      10
Total_visits_bank        6
Total_visits_online     16
Total_calls_made        11
dtype: int64

checking to ensure no duplicated rows

In [14]:
df[df.duplicated(['Customer Key'], keep=False)]

,Customer Key,Avg_Credit_Limit,Total_Credit_Cards,Total_visits_bank,Total_visits_online,Total_calls_made
4,47437,100000,6,0,12,3
48,37252,6000,4,0,2,8
104,97935,17000,2,1,2,10
332,47437,17000,7,3,1,0
391,96929,13000,4,5,0,0
398,96929,67000,6,2,2,2
411,50706,44000,4,5,0,2
432,37252,59000,6,2,1,2
541,50706,60000,7,5,2,2
632,97935,187000,7,1,7,0


From the information above, although there are duplication in the customer key, no row is repeated, so its a separate data

##### Checking for negative values

In [18]:
for i in list(df.columns):
    print  len(df[df[i] < 0]), "negative values in", i
    print "\n"

SyntaxError: invalid syntax (<ipython-input-18-e9998d0c0f2e>, line 2)

Based on the information above, we can see negative values in the Total_visits_online feature/column.
that seems wrong because the minimum value for that variable should be 0

we would handle this error by replacing negative values with zero

In [19]:

# df[df['Total_visits_online'] < 0] = 0

df[df < 0] = 0


checking for neg values again 

In [20]:
for i in list(df.columns):
    print "There are ", len(df[df[i] < 0]), "negative values in", i
    print "\n"

SyntaxError: invalid syntax (<ipython-input-20-4bfc3c30cf71>, line 2)

Next we will take a look at the statistical distribution of the data

In [21]:
df.describe()

,Customer Key,Avg_Credit_Limit,Total_Credit_Cards,Total_visits_bank,Total_visits_online,Total_calls_made
count,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000
mean,55141.443939,34574.242424,4.706061,2.403030,2.606061,3.583333
std,25627.772200,37625.487804,2.167835,1.631813,2.935724,2.865317
min,11265.000000,3000.000000,1.000000,0.000000,0.000000,0.000000
25%,33825.250000,10000.000000,3.000000,1.000000,1.000000,1.000000
50%,53874.500000,18000.000000,5.000000,2.000000,2.000000,3.000000
75%,77202.500000,48000.000000,6.000000,4.000000,4.000000,5.000000
max,99843.000000,200000.000000,10.000000,5.000000,15.000000,10.000000


next we check for outliers

In [22]:
for feature in list(df.columns):
    print "Box plot for " + feature
    sns.boxplot(x=df[feature])
    plt.show()
    print "\n\n"

SyntaxError: Missing parentheses in call to 'print'. Did you mean print("Box plot for " + feature)? (<ipython-input-22-5da8b55a30a4>, line 2)

From the box plot analysis above, we can confirm that: 
    
* Avg_Credit_Limit feature has outliers
* Total_visits_online feature has outliers

Analysis of the body of distribution

In [ ]:
for feature in list(df.columns):
    print "Box plot for " + feature
    sns.distplot(df[feature], kde=True)
    plt.show()
    print "\n\n"

The distribution above also shows that and have outliers

##### Detecting outliers using Skewness

In [ ]:
for column in list(df):
    print column, ":", df[column].skew()
    print

The skewness value should be btw -1 and 1 if the data is normally distributed

From the analysis above, we can confirm that:

* Avg_Credit_Limit feature has outliers because it is skewed
* Total_visits_online feature has outliers because it is skewed

#### Treatment of outliers

We can handle outliers using

a) Log Transformation

b) Replacing Outliers with Central Values such as Median

c) Drop the outliers if there is no significant data loss

I will go with the log transformation strategy

In [ ]:
df['Avg_Credit_Limit'] = np.log(df['Avg_Credit_Limit'])
df['Total_visits_online'] = np.log(df['Total_visits_online'])

In [ ]:
df[df < 0] = 0

# df[df['Total_visits_online'] < 0]

In [ ]:
for column in ['Avg_Credit_Limit', 'Total_visits_online']:
    print column, ":", df[column].skew()
    print

In [ ]:
for feature in ['Avg_Credit_Limit']:
    print "Box plot for " + feature
    sns.boxplot(x=df[feature])
    plt.show()
    print "\n\n"

### Bi-variate analysis

#### Using Pair plot

In [ ]:
sns.pairplot(df)

From the analysis above it seems that there is no strong linear relationship between any of the features/variables, but we will verify by generating a heatmap next

In [ ]:
corr = df.corr()
print corr

In [ ]:
sns.heatmap(corr)

The heatmap shows us that :


* there is a strong positive correlation of 0.59 between Avg_Credit_Limit and Total_Credit_Cards

* there is a strong negative correlation of -0.651 between Total_calls_made and Total_Credit_Cards

* there is a strong negative correlation of -0.632 between Total_visits_online and Total_visits_bank


# 2 Group Data into similar clusters

Now, we will use K-Means clustering to group data based on their attribute. First, we need to determine the optimal number of groups. For that we conduct the knee test to see where the knee happens.

we need to remove the units from each column

In [ ]:
dfScaled=df.apply(zscore)
dfScaledKmeans = dfScaled.copy()
dfScaledKmeans.head()

In [ ]:
dfScaled.head()

In [ ]:
#Finding optimal no. of clusters
from scipy.spatial.distance import cdist
clusters=range(1,10)
meanDistortions=[]

for k in clusters:
    model=KMeans(n_clusters=k)
    model.fit(dfScaled)
    prediction=model.predict(dfScaled)
    meanDistortions.append(sum(np.min(cdist(dfScaled, model.cluster_centers_, 'euclidean'), axis=1)) / dfScaled.shape[0])


plt.plot(clusters, meanDistortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Average distortion')
plt.title('Selecting k with the Elbow Method')

Based on the bend above, 3 or 4 clusters seems like a perfect value

using 3 clusters

In [ ]:
final_model=KMeans(3)
final_model.fit(dfScaled)
prediction=final_model.predict(dfScaled)
prediction_using_3 = prediction

#Append the prediction 
df["GROUP"] = prediction
dfScaled["GROUP"] = prediction
print("Groups Assigned : \n")
df.head()

Analyze the distribution of the data among the two groups (K = 3). One of the most informative visual tool is boxplot

In [ ]:
dfKmeansClust = df.groupby(['GROUP'])
dfKmeansClust.mean()

using 4 clusters

In [ ]:
final_model=KMeans(4)
final_model.fit(dfScaled)
prediction=final_model.predict(dfScaled)
prediction_using_4 = prediction

#Append the prediction 
df["GROUP"] = prediction
dfScaled["GROUP"] = prediction
print("Groups Assigned : \n")
df.head()

Analyze the distribution of the data among the two groups (K = 4). One of the most informative visual tool is boxplot

In [ ]:
dfKmeansClust = df.groupby(['GROUP'])
dfKmeansClust.mean()

In [ ]:
dfScaled.boxplot(by='GROUP', layout = (2,4),figsize=(15,10))

# 3 Hierarchical Clustering

In [ ]:
df=df.iloc[:,:-1]
dfScaled = df.apply(zscore)
dfScaled.head()

In [ ]:
model = AgglomerativeClustering(n_clusters=3, affinity='euclidean',  linkage='average')

In [ ]:
model.fit(dfScaled)

In [ ]:
df['labels'] = model.labels_
df.head(10)

In [ ]:
dfCluster = df.groupby(['labels'])

In [ ]:
dfCluster.mean()

In [ ]:
from scipy.cluster.hierarchy import cophenet, dendrogram, linkage

In [ ]:
from scipy.spatial.distance import pdist  #Pairwise distribution between data points

In [ ]:
# cophenet index is a measure of the correlation between the distance of points in feature space and distance on dendrogram
# closer it is to 1, the better is the clustering

Z = linkage(dfScaled, metric='euclidean', method='average')
c, coph_dists = cophenet(Z , pdist(dfScaled))

c

In [ ]:
plt.figure(figsize=(10, 5))
plt.title('Agglomerative Hierarchical Clustering Dendogram')
plt.xlabel('sample index')
plt.ylabel('Distance')
dendrogram(Z, leaf_rotation=90.,color_threshold = 40, leaf_font_size=8. )
plt.tight_layout()

In [ ]:
# cophenet index is a measure of the correlation between the distance of points in feature space and distance on dendrogram
# closer it is to 1, the better is the clustering

Z = linkage(dfScaled, metric='euclidean', method='complete')
c, coph_dists = cophenet(Z , pdist(dfScaled))

c

In [ ]:
plt.figure(figsize=(10, 5))
plt.title('Agglomerative Hierarchical Clustering Dendogram')
plt.xlabel('sample index')
plt.ylabel('Distance')
dendrogram(Z, leaf_rotation=90.,color_threshold=90,  leaf_font_size=10. )
plt.tight_layout()

In [ ]:
# cophenet index is a measure of the correlation between the distance of points in feature space and distance on dendrogram
# closer it is to 1, the better is the clustering

Z = linkage(dfScaled, metric='euclidean', method='ward')
c, coph_dists = cophenet(Z , pdist(dfScaled))

c

In [ ]:
plt.figure(figsize=(10, 5))
plt.title('Agglomerative Hierarchical Clustering Dendogram')
plt.xlabel('sample index')
plt.ylabel('Distance')
dendrogram(Z, leaf_rotation=90.,color_threshold=600,  leaf_font_size=10. )
plt.tight_layout()

# 4 Silhouette Score

In [ ]:
print 'Silhouette Score(n=3) - KMeans:', silhouette_score(dfScaledKmeans, prediction_using_3)

In [ ]:
print 'Silhouette Score(n=3)- KMeans:', silhouette_score(dfScaledKmeans, prediction_using_4)

In [ ]:
print 'Silhouette Score(n=3)- Agglomerative:', silhouette_score(dfScaled, model.labels_)

The silhouette score seems to be better for the 3 clusters when using kmeans. this suggests that our datasets should have an optimal value of 3 clusters.
In our agglomerative clustering we can see that the clustering is continous and we will have to decide what value 
makes sense for the optimal cluster value.

It is also important to note that we have the best result for the agglomerative clustering using the average linkage method as the cophenet index seems to be the highest amongest all the others

We also have very similar silhouette score for both Kmeans and agglomerative clusters of size 3

As we have seen in the above section, the results of both the clustering are almost similar to the same dataset. It may be possible that when we have a very large dataset, the shape of clusters may differ a little. However, along with many similarities, these two techniques have some differences also.


Some important differences to note:

1) We used "The Elbow method" in the case of KMeans to find the optimal number of clusters while in Hierarchical Clustering we used the Dendrogram

2) KMeans is centriod based while Hierarchical Clustering uses a Hierarchical/agglomerative approach


1. There are 3 different segments of customers

2. The first segment of customers seem to have the highest total online visits. while the second seem to have the highest total calls made. all three customer segment have very close average credit card limit.
